# 4.1 - Procesos asíncronos


![async](images/async.png)




**[Documentación](https://docs.python.org/3/library/asyncio.html)**


**asyncio** es una biblioteca para escribir código [concurrente](https://es.wikipedia.org/wiki/Concurrencia_(inform%C3%A1tica)) utilizando la sintaxis async/await. Se utiliza como base en múltiples frameworks asíncronos de Python y provee un alto rendimiento en redes y servidores web, bibliotecas de conexión de base de datos, colas de tareas distribuidas, etc.

Suele encajar perfectamente para operaciones con límite de E/S y código de red estructurado de alto nivel. Además provee un conjunto de APIs de alto nivel para:

+ ejecutar corutinas de Python de manera concurrente y tener control total sobre su ejecución

+ realizar redes E/S y comunicación entre procesos(IPC)

+ controlar subprocesos

+ distribuir tareas a través de colas

+ sincronizar código concurrente

Adicionalmente, existen APIs de bajo nivel para desarrolladores de bibliotecas y frameworks para:

+ crear y administrar bucles de eventos, los cuales proveen APIs asíncronas para redes, ejecutando subprocesos, gestionando señales del sistema operativo, etc..

+ implementar protocolos eficientes utilizando transportes

+ bibliotecas puente basadas en retrollamadas y código con sintaxis async/wait

$$$$

Nosotros nos enfocaremos en el uso de bucles de eventos para la extracción de datos de la web.


### Hola Mundo

In [1]:
import asyncio

# funcion asincrona
async def saludar():
    print('Holaaa...')
    
    await asyncio.sleep(3)
    
    print('pero que pashhhaaa...')
    
    
await saludar()           # en jupyter
#asyncio.run(saludar())   # en un .py

Holaaa...
pero que pashhhaaa...


### Ejemplo response

Comprobando la respuesta de tres urls.

In [2]:
import requests as req

In [3]:
url='https://s3-eu-west-1.amazonaws.com/'

req.get(url)

<Response [200]>

In [4]:
urls=[
    'https://s3-eu-west-1.amazonaws.com/ih-materials/uploads/data-static/documents/breakfast.jpg',
    'https://s3-eu-west-1.amazonaws.com/ih-materials/uploads/data-static/documents/forbidden',
    'https://s3-eu-west-1.amazonaws.com/ih-materials/uploads/data-static/documents/the-html5-breakfast-site.html'
]

In [5]:
for url in urls:   # de una en una
    print(req.get(url))

<Response [200]>
<Response [403]>
<Response [200]>


In [6]:
# asincrone


async def comprobar():
    
    bucle=asyncio.get_event_loop()   # bucle asincrona
    
    futuros=[]
    
    for url in urls:
        
        promesa=bucle.run_in_executor(None, req.get, url)
        
        futuros.append(promesa)
        
    for res in await asyncio.gather(*futuros):  # aqui se los pido, los response
        print(res.status_code)
        
        
await comprobar()

200
403
200


### Ejemplo ESPN

Volvamos al ejemplo de scrapeo de la págine de ESPN. Vamos a realizar múltiples requests para obtener los datos de todos los equipos.


https://www.espn.com/soccer/competitions

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By

import time

import pandas as pd

from webdriver_manager.chrome import ChromeDriverManager

PATH=ChromeDriverManager().install()

import warnings
warnings.filterwarnings('ignore')

In [8]:
url='https://www.espn.com/soccer/competitions'

In [9]:
driver=webdriver.Chrome(PATH)

driver.get(url)

# cookies
aceptar=driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
aceptar.click()

time.sleep(4)

# selecciona equipos laliga
equipos=driver.find_element(By.XPATH, '//*[@id="fittPageContainer"]/div[3]/div/div[1]/div/div[2]/div[2]/div/div[4]/div/section/div/div/span[2]/a')
equipos.click()

time.sleep(2)

In [10]:
stats=driver.find_elements(By.CSS_SELECTOR,'a.AnchorLink')


teams_stats=[]

for e in stats:
    
    link=e.get_attribute('href')
    
    if '/team/stats/' in link:
        teams_stats.append(link)
        
    else:
        continue
        
        
driver.quit()

teams_stats

['https://www.espn.com/soccer/team/stats/_/id/6832/almeria',
 'https://www.espn.com/soccer/team/stats/_/id/93/athletic-club',
 'https://www.espn.com/soccer/team/stats/_/id/1068/atletico-madrid',
 'https://www.espn.com/soccer/team/stats/_/id/83/barcelona',
 'https://www.espn.com/soccer/team/stats/_/id/85/celta-vigo',
 'https://www.espn.com/soccer/team/stats/_/id/3842/cadiz',
 'https://www.espn.com/soccer/team/stats/_/id/3751/elche',
 'https://www.espn.com/soccer/team/stats/_/id/88/espanyol',
 'https://www.espn.com/soccer/team/stats/_/id/2922/getafe',
 'https://www.espn.com/soccer/team/stats/_/id/9812/girona',
 'https://www.espn.com/soccer/team/stats/_/id/84/mallorca',
 'https://www.espn.com/soccer/team/stats/_/id/97/osasuna',
 'https://www.espn.com/soccer/team/stats/_/id/101/rayo-vallecano',
 'https://www.espn.com/soccer/team/stats/_/id/244/real-betis',
 'https://www.espn.com/soccer/team/stats/_/id/86/real-madrid',
 'https://www.espn.com/soccer/team/stats/_/id/89/real-sociedad',
 'https

In [11]:
len(teams_stats)

20

**Extracción asincrónica**

In [12]:
def asincrono(funcion):
    
    def eventos(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, funcion, *args, **kwargs)
    
    return eventos

In [13]:
DATOS=[]

CABECERAS=[]

In [14]:
@asincrono
def extraer(url):
    
    global DATOS, CABECERAS
    
    # inicia el driver
    driver=webdriver.Chrome(PATH)
    driver.get(url)

    time.sleep(2)

    # acepta cookies
    aceptar=driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
    aceptar.click()

    time.sleep(2)
    
    # disciplina
    dis=driver.find_element(By.XPATH,'//*[@id="fittPageContainer"]/div[2]/div[5]/div/div[1]/section/div/div[2]/nav/ul/li[2]/a')
    dis.click()

    time.sleep(2)
    
    tabla=driver.find_element(By.TAG_NAME,'tbody')

    filas=tabla.find_elements(By.TAG_NAME, 'tr')


    data=[]

    for f in filas:

        elementos=f.find_elements(By.TAG_NAME, 'td') 

        tmp=[]

        for e in elementos:

            tmp.append(e.text)
            
        tmp.append(url.split('/')[-1])
        data.append(tmp)
        

    cabeceras=driver.find_element(By.TAG_NAME, 'thead')

    cabeceras=[c.text for c in cabeceras.find_elements(By.TAG_NAME, 'th')]+['TEAM']
    
    
    DATOS+=data

    CABECERAS=cabeceras

In [15]:
for url in teams_stats[:5]:
    
    res=extraer(url)
    display(res)

<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /Users/iudh/miniforge3/envs/clase/lib/python3.8/asyncio/futures.py:360]>

<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /Users/iudh/miniforge3/envs/clase/lib/python3.8/asyncio/futures.py:360]>

<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /Users/iudh/miniforge3/envs/clase/lib/python3.8/asyncio/futures.py:360]>

<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /Users/iudh/miniforge3/envs/clase/lib/python3.8/asyncio/futures.py:360]>

<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /Users/iudh/miniforge3/envs/clase/lib/python3.8/asyncio/futures.py:360]>

In [20]:
df=pd.DataFrame(DATOS, columns=CABECERAS)

In [21]:
df.head()

,RK,NAME,P,YC,RC,PTS,TEAM
0,1,Sergio Busquets,12,5,1,8,barcelona
1,2,Gerard Piqué,6,3,1,6,barcelona
2,3,Robert Lewandowski,14,0,1,3,barcelona
3,,Franck Kessié,7,3,0,3,barcelona
4,,Ronald Araújo,6,3,0,3,barcelona


In [22]:
df.shape

(119, 7)

In [23]:
df.TEAM.unique()

array(['barcelona', 'athletic-club', 'atletico-madrid', 'celta-vigo',
       'almeria'], dtype=object)